# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *

# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [2]:
path_tmp = '/Users/rince/Desktop/Horse/Data/2020/'

results = load_csv(path_tmp+'results_2020.csv')
horse_results = load_csv(path_tmp+'horse_2020.csv')
peds = load_csv(path_tmp+'peds_2020.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_tmp+'returns_2020.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

In [4]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

In [11]:
len(results)

46968

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [5]:
ll = LearnLGBM(peds,results,horse_results)
ll.path_ft = "/Users/rince/Desktop/Horse/code/horse/peds_ft.txt"
ll.learn_lgb()


pe finish
pe regularizrd


/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 40892
[LightGBM] [Info] Number of data points in the train set: 37228, number of used features: 172


In [10]:
for col in ll.x_train.columns:
    print(col)

枠番
馬番
斤量
course_len
horse_id
jockey_id
年齢
体重
体重変化
n_horse
着順_5R
賞金_5R
着差_5R
first_corner_5R
first_to_rank_5R
first_to_final_5R
final_to_rank_5R
着順_course_len_5R
賞金_course_len_5R
着差_course_len_5R
first_corner_course_len_5R
first_to_rank_course_len_5R
first_to_final_course_len_5R
final_to_rank_course_len_5R
着順_race_type_5R
賞金_race_type_5R
着差_race_type_5R
first_corner_race_type_5R
first_to_rank_race_type_5R
first_to_final_race_type_5R
final_to_rank_race_type_5R
着順_開催_5R
賞金_開催_5R
着差_開催_5R
first_corner_開催_5R
first_to_rank_開催_5R
first_to_final_開催_5R
final_to_rank_開催_5R
着順_9R
賞金_9R
着差_9R
first_corner_9R
first_to_rank_9R
first_to_final_9R
final_to_rank_9R
着順_course_len_9R
賞金_course_len_9R
着差_course_len_9R
first_corner_course_len_9R
first_to_rank_course_len_9R
first_to_final_course_len_9R
final_to_rank_course_len_9R
着順_race_type_9R
賞金_race_type_9R
着差_race_type_9R
first_corner_race_type_9R
first_to_rank_race_type_9R
first_to_final_race_type_9R
final_to_rank_race_type_9R
着順_開催_9R
賞金_開催_9R
着差_開催_9

In [51]:
ll.x_train[ll.x_train['horse_id']==7317]['着順_開催_5R']

202008010106    0.0
202005010209    0.0
202006020109    0.0
202009020609    0.0
202002010112    0.0
202002020208    2.0
202001020408    0.0
202001020807    3.0
202007020708    0.0
Name: 着順_開催_5R, dtype: float64

In [ ]:
# 平均じゃなくて, そのままの値が入っている

202008010106    464
202005010209    462
202006020109    454
202009020609    456
202002010112    460
202002020208    460
202001020408    458
202001020807    456
202007020708    452
Name: 体重, dtype: int64

In [47]:
results.loc[202002020208]

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202002020208,1,2,2,ポリアンサ,牝4,55.0,杉原誠人,2:01.5,NaN,27.2,7.0,482(+6),[東] 木村哲也,2000,曇,芝,良,2020年7月5日,2016104241,1135
202002020208,2,3,3,ルビーカサブランカ,牝3,52.0,藤岡康太,2:02.2,4,2.8,1.0,460(0),[西] 須貝尚介,2000,曇,芝,良,2020年7月5日,2017105582,1116
202002020208,3,7,7,イルマタル,牝4,55.0,藤岡佑介,2:02.5,2,3.8,3.0,466(+24),[西] 渡辺薫彦,2000,曇,芝,良,2020年7月5日,2016105066,1093
202002020208,4,1,1,アークロイヤル,牝4,55.0,岩田康誠,2:02.7,1.1/4,11.2,5.0,422(+2),[西] 今野貞一,2000,曇,芝,良,2020年7月5日,2016104968,5203
202002020208,5,8,9,ウインアルカンナ,牝3,52.0,横山武史,2:03.3,3.1/2,3.0,2.0,462(+10),[東] 武市康男,2000,曇,芝,良,2020年7月5日,2017101417,1170
202002020208,6,4,4,デサフィアンテ,牝5,55.0,丸山元気,2:03.5,1.1/4,9.4,4.0,500(+2),[東] 稲垣幸雄,2000,曇,芝,良,2020年7月5日,2015104642,1127
202002020208,7,5,5,マジックリアリズム,牝4,55.0,大野拓弥,2:03.5,ハナ,23.4,6.0,462(+6),[東] 尾関知人,2000,曇,芝,良,2020年7月5日,2016105091,1096
202002020208,8,6,6,レディーキティ,牝5,55.0,国分恭介,2:03.7,3/4,35.2,8.0,406(-4),[西] 野中賢二,2000,曇,芝,良,2020年7月5日,2015104245,1124
202002020208,9,7,8,レローヴ,牝6,55.0,横山和生,2:04.1,2.1/2,83.8,9.0,510(0),[東] 黒岩陽一,2000,曇,芝,良,2020年7月5日,2014105371,1140
202002020208,10,8,10,ミフリマ,牝4,53.0,亀田温心,2:04.5,2.1/2,248.2,10.0,452(+4),[東] 本間忍,2000,曇,芝,良,2020年7月5日,2016103653,1176


In [46]:
ll.x_train.loc[ll.y_train.index[4]]

,枠番,馬番,斤量,course_len,horse_id,jockey_id,年齢,体重,体重変化,n_horse,...,race_type_芝,race_type_ダート,race_type_障害,ground_state_良,ground_state_稍重,ground_state_重,ground_state_不良,性_牝,性_牡,性_セ
202008010106,3,5,54.0,18.0,7317,56,3,464,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,2,3,54.0,18.0,5563,13,3,418,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,2,4,56.0,18.0,7329,21,3,496,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,5,9,56.0,18.0,4228,82,3,490,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,4,8,54.0,18.0,4881,91,3,442,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,5,10,52.0,18.0,7558,124,3,468,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,8,15,54.0,18.0,5520,6,3,460,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,1,1,56.0,18.0,4380,61,3,484,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,7,13,56.0,18.0,7545,136,3,440,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,7,14,56.0,18.0,5676,80,3,448,0,16,...,1,0,0,1,0,0,0,0,1,0


In [41]:
ll.y_train.loc[ll.y_train.index[8]]

202008010106    1
202008010106    1
202008010106    1
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
202008010106    0
Name: rank, dtype: int64

In [36]:
ll.y_train.loc[202002020208]

202002020208    1
202002020208    1
202002020208    1
202002020208    0
202002020208    0
202002020208    0
202002020208    0
202002020208    0
202002020208    0
202002020208    0
Name: rank, dtype: int64

# デバグ

In [ ]:
x_train = ll.x_train
y_train = ll.y_train
x_test = ll.x_test
y_test = ll.y_test

In [71]:
for col in x_train.columns:
    print(col)

枠番
馬番
斤量
course_len
horse_id
jockey_id
年齢
体重
体重変化
n_horse
着順_5R
賞金_5R
着差_5R
first_corner_5R
first_to_rank_5R
first_to_final_5R
final_to_rank_5R
着順_course_len_5R
賞金_course_len_5R
着差_course_len_5R
first_corner_course_len_5R
first_to_rank_course_len_5R
first_to_final_course_len_5R
final_to_rank_course_len_5R
着順_race_type_5R
賞金_race_type_5R
着差_race_type_5R
first_corner_race_type_5R
first_to_rank_race_type_5R
first_to_final_race_type_5R
final_to_rank_race_type_5R
着順_開催_5R
賞金_開催_5R
着差_開催_5R
first_corner_開催_5R
first_to_rank_開催_5R
first_to_final_開催_5R
final_to_rank_開催_5R
着順_9R
賞金_9R
着差_9R
first_corner_9R
first_to_rank_9R
first_to_final_9R
final_to_rank_9R
着順_course_len_9R
賞金_course_len_9R
着差_course_len_9R
first_corner_course_len_9R
first_to_rank_course_len_9R
first_to_final_course_len_9R
final_to_rank_course_len_9R
着順_race_type_9R
賞金_race_type_9R
着差_race_type_9R
first_corner_race_type_9R
first_to_rank_race_type_9R
first_to_final_race_type_9R
final_to_rank_race_type_9R
着順_開催_9R
賞金_開催_9R
着差_開催_9

In [75]:
x_train


,枠番,馬番,斤量,course_len,horse_id,jockey_id,年齢,体重,体重変化,n_horse,...,race_type_芝,race_type_ダート,race_type_障害,ground_state_良,ground_state_稍重,ground_state_重,ground_state_不良,性_牝,性_牡,性_セ
202008010106,3,5,54.0,18.0,7317,56,3,464,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,2,3,54.0,18.0,5563,13,3,418,0,16,...,1,0,0,1,0,0,0,1,0,0
202008010106,2,4,56.0,18.0,7329,21,3,496,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,5,9,56.0,18.0,4228,82,3,490,0,16,...,1,0,0,1,0,0,0,0,1,0
202008010106,4,8,54.0,18.0,4881,91,3,442,0,16,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202008040210,4,7,55.0,12.0,2627,80,4,470,-6,16,...,0,1,0,0,0,1,0,1,0,0
202008040210,5,10,55.0,12.0,2936,136,4,504,4,16,...,0,1,0,0,0,1,0,1,0,0
202008040210,6,12,57.0,12.0,892,15,6,452,6,16,...,0,1,0,0,0,1,0,0,1,0
202008040210,3,6,57.0,12.0,619,99,6,502,-10,16,...,0,1,0,0,0,1,0,0,1,0


In [85]:
x_train.loc[202008040210]
""" 
horse_id = 5408
この馬に着目する

"""

x_train[x_train['horse_id']==5408]['着順_開催_allR']

202009010106    1.500000
202009030112    1.333333
202004020111    0.000000
202008040210    0.000000
Name: 着順_開催_allR, dtype: float64

In [84]:
x_test[x_test['horse_id']==5408]

,枠番,馬番,斤量,course_len,horse_id,jockey_id,年齢,体重,体重変化,n_horse,...,race_type_芝,race_type_ダート,race_type_障害,ground_state_良,ground_state_稍重,ground_state_重,ground_state_不良,性_牝,性_牡,性_セ


In [ ]:
x_train

In [34]:
ll.x_test[ll.x_test['horse_id']==7317]['着順_開催_allR']

202007030112    4.0
202007030608    3.0
Name: 着順_開催_allR, dtype: float64

In [28]:
ll.x_train[ll.x_train['horse_id']==9992]['着順_開催_allR']

202006040704    0.0
Name: 着順_開催_allR, dtype: float64

In [31]:
ll.y_test.loc[202006050801]

202006050801    1
202006050801    1
202006050801    1
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
202006050801    0
Name: rank, dtype: int64

In [29]:
#ll.x_test['着順_開催_allR'].loc[202006050801] # 4.00
print(ll.x_test.loc[202006050801].iloc[0])
9992


枠番                    5
馬番                    9
斤量                   54
course_len           12
horse_id           9992
                   ... 
ground_state_重        0
ground_state_不良       0
性_牝                   1
性_牡                   0
性_セ                   0
Name: 202006050801, Length: 172, dtype: object


9992

In [ ]:
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [ ]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()


val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [76]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(pt.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
# val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [84]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                # 'lambdarank_truncation_level': 37,
                #  'learning_rate': 0.01271180569095827,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'n_estimators': 97,
                'num_leaves': 42,
                'force_col_wise':True
            }



In [ ]:
import warnings

warnings.filterwarnings("ignore")
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [67]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        # 'num_iterations': hp.choice('num_iterations',range(50,120)),
        # 'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [ ]:
trials = Trials()
optimize(trials)

# Simulate


In [91]:
length = len(ll.r.data_c)
test_c = ll.r.data_c.iloc[int(0.8*length):]

In [92]:

for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    sl.calc_tansho(test_c,return_tables,odds=odds)




odds 1.1
not_bet_count 0
---------------------
単勝
的中率 : 193 / 662
的中% : 29.15 %
収支   : 17780.0 円
的中レース : {'01': 19, '02': 19, '03': 20, '04': 11, '05': 9, '06': 14, '07': 17, '08': 19, '09': 15, '10': 20, '11': 15, '12': 15}

odds 2.0
not_bet_count 77
---------------------
単勝
的中率 : 153 / 585
的中% : 26.15 %
収支   : 18820.0 円
的中レース : {'01': 19, '02': 19, '03': 20, '04': 11, '05': 9, '06': 14, '07': 17, '08': 19, '09': 15, '10': 20, '11': 15, '12': 15}

odds 3.0
not_bet_count 201
---------------------
単勝
的中率 : 98 / 461
的中% : 21.26 %
収支   : 18080.0 円
的中レース : {'01': 19, '02': 19, '03': 20, '04': 11, '05': 9, '06': 14, '07': 17, '08': 19, '09': 15, '10': 20, '11': 15, '12': 15}

odds 4.0
not_bet_count 323
---------------------
単勝
的中率 : 62 / 339
的中% : 18.29 %
収支   : 18260.0 円
的中レース : {'01': 19, '02': 19, '03': 20, '04': 11, '05': 9, '06': 14, '07': 17, '08': 19, '09': 15, '10': 20, '11': 15, '12': 15}

odds 5.0
not_bet_count 418
---------------------
単勝
的中率 : 47 / 244
的中% : 19.26 %
収支   : 2126

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

# race_id_list

In [93]:

202205021011
race_id_list = ['2022040301{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100401{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010201{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Predictor

In [97]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
pt.learn_lgb(test_size=0)

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.args, arg0: int) -> None

Invoked with: <fasttext_pybind.args object at 0x7faf92277fb8>, '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'

In [111]:
pt.model_ft

In [114]:
pe_tmp = Peds(pt.peds)
pe_tmp.regularize_peds() 

In [117]:
pe_tmp.peds_re
path_ft = pt.path_ft
pe_tmp.peds_re.to_csv(path_ft,header=False,index=False,sep=',')

In [110]:
train,test = pt.get_train_data()
# pt.process_hr(results,horse_results)
# pt.learn_model_ft()

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.args, arg0: int) -> None

Invoked with: <fasttext_pybind.args object at 0x7faf93fc7d50>, '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'

In [108]:
pt.process_pe(peds)


pe finish
pe regularizrd


In [109]:
pt.process_hr(results,horse_results)


hr finish


# 東京

In [120]:
base  = '2022050304'
pt.predict(base+'10')


              馬番    scores
202205030410   6  1.371102
202205030410   3  0.779867
202205030410   8  0.443859
202205030410   2  0.121382
202205030410   5 -0.451744
202205030410  14 -0.532018
202205030410  10 -0.581835
202205030410  11 -0.688470
202205030410   4 -0.755791
202205030410   9 -0.825304
202205030410  12 -0.914896
202205030410  13 -0.947285
202205030410  15 -1.005432
202205030410   1 -1.042639
202205030410   7 -1.054250


# 中京

In [119]:
base  = '2022070404'
# for i in ['07','08','09','10','11']:
pt.predict(base+'10')


              馬番    scores
202207040410  10  0.893336
202207040410   2  0.273061
202207040410   4 -0.812102
202207040410   8 -0.930210
202207040410   7 -0.981755
202207040410   9 -1.017185
202207040410   5 -1.021039
202207040410   1 -1.025512
202207040410   6 -1.042639
202207040410   3 -1.066951


# 函館

In [ ]:
base  = '2022020102'
# for i in ['07','08','09','10','11']:
pt.predict(base+'11')

In [ ]:

pt.show_results_today()

# 日付に注意

In [4]:
date = '2022/12/31'

# Results scraping

In [5]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

  0%|          | 0/144 [00:00<?, ?it/s]

# Horse_results scraping

In [6]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/1902 [00:00<?, ?it/s]

# Peds scraping

In [7]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/1902 [00:00<?, ?it/s]

  0%|          | 0/1902 [00:00<?, ?it/s]

# Return scraping

In [8]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/144 [00:00<?, ?it/s]

# update

In [9]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [10]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価